In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats
import seaborn as sns

In [2]:
#Make the normal curve for the given data
office1 = stats.norm(90, 15)
office2 = stats.norm(100, 20)

In [9]:
#We're comparing two groups, so we'll nead there means and std-devs.
xbar1 = office1.mean()
xbar2 = office2.mean()

#Trial size was given in question
n1 = 40
n2 = 50

s1 = office1.std()
s2 = office2.std()

In [10]:
#Taking a little off
degf = n1 + n2 - 2

In [11]:
#This one is a doozy.
s_p = sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))

In [12]:
#This might be unecessary for this question.
standard_error = se = sqrt(s1**2 / n1 + s2**2 / n2)

In [13]:
#This is the score we're going to find out.
#This is that crazy long 2-Sample T Test formula written out in Python
t = (xbar1 - xbar2) / (s_p * sqrt(1/n1 + 1/n2))


In [14]:
t

-2.6252287036468456

In [17]:
#more than .99 of t-values are higher than this t score
stats.t(degf).sf(t)

0.9948950737753803

In [18]:
stats.t(degf).cdf(t)

0.005104926224619695

In [19]:
stats.t(degf).pdf(t)

0.013882685754167083

In [ ]:
#Got these from an online 2-sample T-Test app
# Equal Variances
# Pooled Standard Deviation: 17.9567
# Pooled DF: 88
# 95% Confidence Interval for the Difference ( -17.3355 , -2.6645 )

# Test Statistic t= -2.6252
# Population 1 ≠ Population 2: P-Value = 0.0102
# Population 1 < Population 2: P-Value = 0.9949
# Population 1 > Population 2: P-Value = 0.0051